# Evaluate Product Sales

Exploratory exercises over the product orders data for Wide World Importers

- What are the top 20 most popular products?
- How do sales of the most popular product vary over time?
- Is there a pattern to these sales?
 
## What are the most popular products?
 Load the libraries required to perform these tasks.


In [1]:
library(AzureML)
library(dplyr)
library(ggplot2)
library(scales)

install.packages("xts") # You also need to install the xts package
library(xts)

install.packages("broom") # And the broom package
library(broom)


Attaching package: âdplyrâ

The following objects are masked from âpackage:statsâ:

    filter, lag

The following objects are masked from âpackage:baseâ:

    intersect, setdiff, setequal, union

Installing package into â/home/nbuser/Râ
(as âlibâ is unspecified)
Loading required package: zoo

Attaching package: âzooâ

The following objects are masked from âpackage:baseâ:

    as.Date, as.Date.numeric


Attaching package: âxtsâ

The following objects are masked from âpackage:dplyrâ:

    first, last

The following object is masked from âpackage:AzureMLâ:

    endpoints

Installing package into â/home/nbuser/Râ
(as âlibâ is unspecified)


In [2]:
library("AzureML")
ws <- workspace()
ordersDataSet <- download.datasets(ws, "Wide World Importers")
head(ordersDataSet)

StockItemID,Description,Quantity,UnitPrice,PickingCompletedWhen
164,32 mm Double sided bubble wrap 50m,50,112,1/2/2013 11:00:00 AM
67,Ride on toy sedan car (Black) 1/12 scale,10,230,1/1/2013 11:00:00 AM
50,Developer joke mug - old C developers never die (White),9,13,1/1/2013 11:00:00 AM
89,The Gu red shirt XML tag t-shirt (Black) 3XS,72,18,1/2/2013 11:00:00 AM
171,32 mm Anti static bubble wrap (Blue) 10m,90,32,1/2/2013 11:00:00 AM
10,USB food flash drive - chocolate bar,9,32,1/1/2013 11:00:00 AM


The same product might occur in more than one order. Generate a list of products and their descriptions that have been ordered at least once. Each product should occur only once in the list. Store the result in another dataset called *stockItemsDataSet*.

In [3]:
ordersDataSet %>%
  select(StockItemID, Description) %>%
  distinct() -> stockItemsDataSet

head(stockItemsDataSet)

StockItemID,Description
164,32 mm Double sided bubble wrap 50m
67,Ride on toy sedan car (Black) 1/12 scale
50,Developer joke mug - old C developers never die (White)
89,The Gu red shirt XML tag t-shirt (Black) 3XS
171,32 mm Anti static bubble wrap (Blue) 10m
10,USB food flash drive - chocolate bar


Find which products are the most popular, using a *dplyr* pipeline as follows:

- Extract the StockItemID and quantity sold for each order
- Group the results by StockItemID
- Sum the quantity sold for each group, and name the column as *numOrdered*
- Order the results in descending order of *numOrdered* (the most popular items will be at the top of the list)
- Limit the results to the first 20 rows

Save the results in another dataset named *numOrderedDataSet*

In [4]:
ordersDataSet %>%
  select(StockItemID, Quantity) %>%
  group_by(StockItemID) %>%
  summarize(numOrdered = sum(Quantity)) %>% 
  arrange(desc(numOrdered)) %>%
  filter(row_number() <= 20) -> numOrderedDataSet

head(numOrderedDataSet)

StockItemID,numOrdered
191,207324
192,193680
189,158626
188,152375
185,152125
184,151875


Combine the *numOrderedDataSet* and the *stockItemsDataSet* over the *StockItemID* column so that the list contains the *StockItemID*, *numOrdered*, and *Description* columns

In [5]:
results <- inner_join(numOrderedDataSet, stockItemsDataSet, by = "StockItemID")

head(results)

StockItemID,numOrdered,Description
191,207324,Black and orange fragile despatch tape 48mmx75m
192,193680,Black and orange fragile despatch tape 48mmx100m
189,158626,Clear packaging tape 48mmx75m
188,152375,3 kg Courier post bag (White) 300x190x95mm
185,152125,Shipping carton (Brown) 356x356x279mm
184,151875,Shipping carton (Brown) 305x305x305mm


Generate a graph showing the product descriptions on the X-axis and the number ordered on the Y-axis

In [6]:
options(repr.plot.width=10, repr.plot.height=8)  
ggplot(results) +
  geom_point(mapping = aes(x = reorder(Description, -numOrdered), y = numOrdered), color = "red", size = 5) +
  labs(x = "Product", y = "Total Ordered") +
  scale_x_discrete(labels = function(x) { lapply(strwrap(x, width = 25, simplify = FALSE), paste, collapse = "\n")}) +
  theme(axis.text.x = element_text(angle = 90, size = 10))

## How do product sales vary over time?

Product 191 (Black and orange fragile despatch tape 48mmx75m) is the most popular product. How do sales of this product vary over time (if at all). 

Find all sales of product 191.

In [7]:
ordersDataSet %>%
  filter(StockItemID == 191) -> productSalesDataSet

head(productSalesDataSet)
str(productSalesDataSet)

StockItemID,Description,Quantity,UnitPrice,PickingCompletedWhen
191,Black and orange fragile despatch tape 48mmx75m,288,3,1/2/2013 11:00:00 AM
191,Black and orange fragile despatch tape 48mmx75m,144,3,1/2/2013 11:00:00 AM
191,Black and orange fragile despatch tape 48mmx75m,252,3,1/2/2013 11:00:00 AM
191,Black and orange fragile despatch tape 48mmx75m,108,3,1/2/2013 11:00:00 AM
191,Black and orange fragile despatch tape 48mmx75m,180,3,1/3/2013 11:00:00 AM
191,Black and orange fragile despatch tape 48mmx75m,180,3,1/4/2013 11:00:00 AM


'data.frame':	1040 obs. of  5 variables:
 $ StockItemID         : int  191 191 191 191 191 191 191 191 191 191 ...
 $ Description         : chr  "Black and orange fragile despatch tape 48mmx75m" "Black and orange fragile despatch tape 48mmx75m" "Black and orange fragile despatch tape 48mmx75m" "Black and orange fragile despatch tape 48mmx75m" ...
 $ Quantity            : int  288 144 252 108 180 180 252 72 324 216 ...
 $ UnitPrice           : int  3 3 3 3 3 3 3 3 3 3 ...
 $ PickingCompletedWhen: chr  "1/2/2013 11:00:00 AM" "1/2/2013 11:00:00 AM" "1/2/2013 11:00:00 AM" "1/2/2013 11:00:00 AM" ...


Reorganize the data as a time series, based on the *PickingCompletedWhen* column. Note that this column is actually a string (chr) in month/day/year format, so it needs to be converted to a datetime.

In [8]:
timeSeriesOrders <- xts(productSalesDataSet[c("StockItemID", "Quantity")], 
                        order.by = as.POSIXct(productSalesDataSet$PickingCompletedWhen, format = "%m/%d/%Y"))
head(timeSeriesOrders)

           StockItemID Quantity
2013-01-02         191      288
2013-01-02         191      144
2013-01-02         191      252
2013-01-02         191      108
2013-01-03         191      180
2013-01-04         191      180

Work out how many units where ordered each day.

In [9]:
dailyTotals <- apply.daily(timeSeriesOrders, function(x){ apply(x$Quantity, 2, sum)})

head(dailyTotals)

           [,1]
2013-01-02  792
2013-01-03  180
2013-01-04  180
2013-01-10  324
2013-01-11  324
2013-01-14  216

Plot a graph showing how the sales vary by day.

First, the time series must be converted back into a dataframe to work with ggplot.

In [10]:
graphData <- tidy(dailyTotals)
head(graphData)

Warning message in data.row.names(row.names, rowsi, i):
âsome row.names duplicated: 2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,

index,series,value
2013-01-02,x,792
2013-01-03,x,180
2013-01-04,x,180
2013-01-10,x,324
2013-01-11,x,324
2013-01-14,x,216


Then convert the index column back into a datetime value.

In [11]:
graphData$index <- as.POSIXct(graphData$index, origin="1970-01-01")
head(graphData)

index,series,value
2013-01-02,x,792
2013-01-03,x,180
2013-01-04,x,180
2013-01-10,x,324
2013-01-11,x,324
2013-01-14,x,216


Finally, plot the data.

In [12]:
options(repr.plot.width=15, repr.plot.height=5)
ggplot(graphData, aes(x = index, y = value)) +
  ggtitle("Product Sales Over Time for Product 191") +
  xlab("Date") +
  ylab("Units Sold") +
  scale_x_datetime(labels = date_format("%Y-%m"), breaks = date_breaks("months")) +
  theme(axis.text.x = element_text(angle = 45)) +
  geom_point(color = "blue")

## Is there a pattern to these sales?

Focus on sales for 2013.

In [13]:
firstDay <- as.POSIXct("2013-01-01")
lastDay <- as.POSIXct("2013-12-31")
day1 <- as.numeric(firstDay)
dayN <- as.numeric(lastDay)
productSalesFor2013 <- filter(graphData, (index >= day1) & (index <= dayN))

options(repr.plot.width=15, repr.plot.height=5)
ggplot(productSalesFor2013, aes(x = index, y = value)) +
  ggtitle("Product Sales for Product 191 in 2013") +
  xlab("Date") +
  ylab("Units Sold") +
  scale_x_datetime(labels = date_format("%Y-%m"), breaks = date_breaks("months")) +
  theme(axis.text.x = element_text(angle = 45)) +
  geom_point(color = "blue")

Fit a curve to the sales, and then compare to curves for 2014 and 2015 to see if there is a regular pattern.

In [14]:
ggplot(productSalesFor2013, aes(x = index, y = value)) +
  ggtitle("Product Sales for Product 191 in 2013") +
  xlab("Date") +
  ylab("Units Sold") +
  scale_x_datetime(labels = date_format("%Y-%m"), breaks = date_breaks("months")) +
  theme(axis.text.x = element_text(angle = 45)) +
  geom_point(color = "black", alpha = 0.2) +
  geom_smooth(method = "lm", formula = y ~ poly(x, 9))

The graph and curve for 2014.

In [15]:
firstDay <- as.POSIXct("2014-01-01")
lastDay <- as.POSIXct("2014-12-31")
day1 <- as.numeric(firstDay)
dayN <- as.numeric(lastDay)
productSalesFor2014 <- filter(graphData, (index >= day1) & (index <= dayN))

ggplot(productSalesFor2014, aes(x = index, y = value)) +
  ggtitle("Product Sales for Product 191 in 2014") +
  xlab("Date") +
  ylab("Units Sold") +
  scale_x_datetime(labels = date_format("%Y-%m"), breaks = date_breaks("months")) +
  theme(axis.text.x = element_text(angle = 45)) +
  geom_point(color = "black", alpha = 0.2) +
  geom_smooth(method = "lm", formula = y ~ poly(x, 9))

The graph and curve for 2015.

In [16]:
firstDay <- as.POSIXct("2015-01-01")
lastDay <- as.POSIXct("2015-12-31")
day1 <- as.numeric(firstDay)
dayN <- as.numeric(lastDay)
productSalesFor2015 <- filter(graphData, (index >= day1) & (index <= dayN))

ggplot(productSalesFor2015, aes(x = index, y = value)) +
  ggtitle("Product Sales for Product 191 in 2015") +
  xlab("Date") +
  ylab("Units Sold") +
  scale_x_datetime(labels = date_format("%Y-%m"), breaks = date_breaks("months")) +
  theme(axis.text.x = element_text(angle = 45)) +
  geom_point(color = "black", alpha = 0.2) +
  geom_smooth(method = "lm", formula = y ~ poly(x, 9))